In [25]:
import streamlit as st
import altair as alt
from pathlib import Path
import pandas as pd

from search_models import *
from preprocessing_helpers import *
from scraping_helpers import *
from update_dataset import *

In [26]:
# Construct the relative path to your data file
path = "../data/processed/*.csv"
df = load_df(path)
df.head()



2024-08-01 09:36:12.501 
  command:

    streamlit run /Users/laurent/anaconda3/envs/ds_final_project/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


,title,href,text,time_author,headline,comment_authors,comments,occurrence,url,author,created,updated,normalized_text,cities,countries,flight_phase,from,to
0,"Delta A333 near Goose Bay on Jul 25th 2024, en...",/h?article=51bbef52&opt=0,"A Delta Airlines Airbus A330-300, registration...","By Simon Hradecky, created Monday, Jul 29th 20...",Incident: Delta A333 near Goose Bay on Jul 25t...,"['By Jetman on Tuesday, Jul 30th 2024 06:31Z',...",['With resume service in 3 hrs in remote locat...,incident,https://avherald.com/h?article=51bbef52&opt=0,Simon Hradecky,2024-07-29 19:21:00,2024-07-29 19:21:00,airlines airbus registration performing flight...,"{'detroit', 'bay', 'amsterdam', 'delta', 'atla...","{'netherlands', 'canada'}",enroute,erd,oi
1,"Frontier A20N near Denver on Jul 28th 2024, st...",/h?article=51bbedd8&opt=0,"A Frontier Airlines Airbus A320-200N, registra...","By Simon Hradecky, created Monday, Jul 29th 20...",Incident: Frontier A20N near Denver on Jul 28t...,"['By DJH on Tuesday, Jul 30th 2024 04:57Z', 'B...",['The article is not clear to me. Does this re...,incident,https://avherald.com/h?article=51bbedd8&opt=0,Simon Hradecky,2024-07-29 19:09:00,2024-07-29 19:09:00,frontier airlines airbus registration performi...,"{'washington', 'denver', 'virginia', 'arlington'}",set(),enroute,denver,wa
2,"United B738 over Pacific on Jul 27th 2024, unr...",/h?article=51bbebe0&opt=0,"A United Boeing 737-800, registration N35260 p...","By Simon Hradecky, created Monday, Jul 29th 20...",Incident: United B738 over Pacific on Jul 27th...,[],[],incident,https://avherald.com/h?article=51bbebe0&opt=0,Simon Hradecky,2024-07-29 18:53:00,2024-07-29 18:53:00,united boeing registration performing flight d...,{'tokyo'},"{'guam', 'japan'}",enroute,NaN,apan
3,"ANA DH8D at Osaka on Jul 28th 2024, engine shu...",/h?article=51bbea80&opt=0,An ANA All Nippon Airways de Havilland Dash 8-...,"By Simon Hradecky, created Monday, Jul 29th 20...","Incident: ANA DH8D at Osaka on Jul 28th 2024, ...",[],[],incident,https://avherald.com/h?article=51bbea80&opt=0,Simon Hradecky,2024-07-29 18:42:00,2024-07-29 18:42:00,ana nippon airways havilland dash registration...,"{'itami', 'matsuyama', 'osaka'}",{'japan'},climb,osa,tsu
4,"Atlas B744 over Pacific on Jul 28th 2024, burn...",/h?article=51bbe950&opt=0,"An Atlas Air Boeing 747-400, registration N487...","By Simon Hradecky, created Monday, Jul 29th 20...",Incident: Atlas B744 over Pacific on Jul 28th ...,"['By (anonymous) on Tuesday, Jul 30th 2024 00:...","[""Let's hope 487 is OK.""]",incident,https://avherald.com/h?article=51bbe950&opt=0,Simon Hradecky,2024-07-29 18:32:00,2024-07-29 18:32:00,atlas air boeing registration performing fligh...,"{'anchorage', 'chitose', 'sapporo', 'seoul'}",{'japan'},enroute,seoul,ho


In [56]:
# Get list of all airport cities

import airportsdata
airports = airportsdata.load('IATA')  # key is the IATA location code
airports["ZRH"]["city"]

airport_cities = []
for airport in airports:
    city = airports[airport]["city"]
    airport_cities.append(city.lower())

airport_cities[:20]

['key largo',
 'yakataga',
 'farewell',
 'crested butte',
 'columbus',
 'johnson city',
 'icy bay',
 'hogatza',
 'bush',
 'blakely island',
 'kenai',
 'bradford',
 'calverton',
 'rolling fork',
 'belle chasse',
 'center island',
 'corolla',
 'stuart island',
 'fort bragg',
 'mullen']

In [32]:
for i in range(20):
    text = df.iloc[i].text
    print(text)

A Delta Airlines Airbus A330-300, registration N830NW performing flight DL-137 from Amsterdam (Netherlands) to Detroit,MI (USA), was enroute at FL380 about 140nm northeast of Goose Bay,NL (Canada) when the crew detected oil quantity and pressure indications for the left hand engine (CF6) dropped. The crew worked the related checklists and shut the engine down, subsequently diverted to Goose Bay for a safe landing. The Canadian TSB reported maintenance detected an oil leak around the accessory section, then noticed that the starter mount had fractured into several pieces, there was also damage to two oil lines. The damage is being assessed.The aircraft remained on the ground for about 3 days, then positioned to Atlanta,GA (USA) and resumed service.https://flightaware.com/live/flight/DAL137/history/20240725/1145Z/EHAM/KDTW
A Frontier Airlines Airbus A320-200N, registration N383FR performing flight F9-688 from Denver,CO to Washington Dulles,DC (USA), was enroute at FL350 about 650nm eastn

In [49]:
def get_from_airport(text, city_names):
    if len(text) > 300:
        sentence = text[:300].lower()
    else:
        sentence = text.lower()

    # Detect the first "from" if any
    sentence = sentence.split("from")

    if len(sentence) > 1:
        # Take the part of the sentence after "from"
        sentence = sentence[1].strip()

        # Detect "to" if any
        sentence = sentence.split("to")

        if len(sentence) > 1:
            from_ = sentence[0]
            to_ = sentence[1].split(")")[0].strip()
            print(from_)
            print(to_)
            from_city = "dummy from"
            to_city = "dummy to"

            for city_name in city_names:
                if city_name in from_:
                    from_city = city_name
                    print(from_city)
                if city_name in to_:
                    to_city = city_name
                    print(to_city)

            return from_city, to_city
        
        else:
            return None, None     
    
    else:
        return None, None

In [57]:
text = "A Delta Airlines Airbus A330-300, registration N830NW performing flight DL-137 from yakataga (Netherlands) to calverton,MI (USA), was enroute at FL380 about 140nm northeast of Goose Bay,NL (Canada) when the crew detected oil quantity and pressure indications for the left hand engine (CF6) dropped. The crew worked the related checklists and shut the engine down, subsequently diverted to Goose Bay for a safe landing. The Canadian TSB repor"

print(get_from_airport(text, airport_cities[:20]))

yakataga (netherlands) 
calver
yakataga
('yakataga', 'dummy to')


In [34]:
for i in range(20):
    text = df.iloc[i].text
    print(get_from_airport(text, airport_cities))

('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
